<a href="https://colab.research.google.com/github/timsetsfire/wandb-examples/blob/main/colab/Tensorboard_%2B_W%26B_Sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -qqq

In [ ]:
import torch
import wandb
from torch.utils.tensorboard import SummaryWriter

In [ ]:
run = wandb.init(project = "load-artifact")
art = wandb.Artifact(name = "file2", type = "file-folder")
art.add_dir("./file-folder")
run.log_artifact(art)

wandb: Adding directory to artifact (./file-folder)... Done. 0.1s


In [ ]:
run.finish()

In [ ]:
sweep_config = {
  'method': 'grid', 
  'metric': {
      'name': 'Loss/train',  ## matches what i write via SummaryWriter
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'learning_rate':{
          'values': [0.05,0.025,0.01,0.005,0.001]
      }
  }
}


x = torch.arange(-5, 5, 0.1).view(-1, 1)
y = -5 * x + 0.1 * torch.randn(x.size())


wandb.login()
wandb.tensorboard.patch(root_logdir = "/content/runs") ## since i don't have tensorboard running, i set the root logging directory manually


def sweep_train(config_defaults = None): 
    config_defaults = { "learning_rate": 0.01 }  ## will be overwritten during sweeps
    run = wandb.init(config=config_defaults, sync_tensorboard=True)  # the config gets over-written in the Sweep
    model = torch.nn.Linear(1, 1)
    writer = SummaryWriter(log_dir = "/content/runs")
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = wandb.config.learning_rate)
    for epoch in range(5):
        y1 = model(x)
        loss = criterion(y1, y)
        writer.add_scalar("Loss/train", loss, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    writer.flush()
    writer.close()
    run.finish()


sweep_id = wandb.sweep(sweep_config, project="sweeps-pytorch-tensorboard-v4")
wandb.agent(sweep_id, function=sweep_train)


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: uouzjuts
Sweep URL: https://wandb.ai/tim-w/sweeps-pytorch-tensorboard-v4/sweeps/uouzjuts


wandb: Agent Starting Run: euw6sdz1 with config:
wandb: 	learning_rate: 0.05
wandb: Currently logged in as: tim-w. Use `wandb login --relogin` to force relogin


Loss/train,█▁▁▁▁
global_step,▁▃▅▆█
Loss/train,0.07691
global_step,4


...

wandb: Agent Starting Run: 50kwk73g with config:
wandb: 	learning_rate: 0.005


Loss/train,█▆▄▂▁
global_step,▁▃▅▆█
Loss/train,82.608
global_step,4


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 5wdzodr8 with config:
wandb: 	learning_rate: 0.001


Loss/train,█▆▄▃▁
global_step,▁▃▅▆█
Loss/train,129.18927
global_step,4


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
